## Predict Expected generation 
Use moving average, seasonal features, lag features, and cyclic encoding for hour of day. use other statistical model as needed.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import plot_importance
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

/Users/anupbh/miniconda3/envs/hackathon/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def run_prophet(df):
    '''
    Prophet automatically handles seasonality (daily, yearly, etc.) and trends.
    We do not need to include engineered features like lags, rolling averages, or cyclic encodings in the input DataFrame for Prophet.
    '''
    df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
    prophet_data = df[['Date','Gen']].rename(columns={'Date':'ds','Gen':'y'})
    prophet = Prophet(daily_seasonality=True, yearly_seasonality=True)
    prophet.fit(prophet_data)
    forecast = prophet.predict(prophet_data)

    df['prophet_pred'] = forecast['yhat']  # seasonality + trend baseline
    df['residual'] = df['Gen'] - df['prophet_pred']
    return df


In [3]:
def run_xgboost(df):
    from xgboost import XGBRegressor
    
    res_features = ['HE','is_weekend','P/OP', 'is_vacation']
    X = df[res_features]
    y = df['residual']
    # y = df['Gen']

    # test-train split
    split_index = int(len(df) * 0.8)
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:] 
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
    # y_train_adj = np.where(y_train == 0, 1e-6, y_train)

    reg = XGBRegressor(
        objective='binary:logistic',
        learning_rate=0.05,
        n_estimators=500
    )

    reg.fit(X_train, y_train)
    df.loc[X_test.index, 'xgb_pred'] = reg.predict(X_test)
    df['final_pred'] = df['prophet_pred'] + df['xgb_pred']

    # find accuracy for XGBoost residual model
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    mae = mean_absolute_error(df.loc[X_test.index, 'Gen'], df.loc[X_test.index, 'final_pred'])
    print(f"MAE: {mae}")
    
    return df, reg


## Predict for next 5 years
Expected generation (by month, peak and off-peak periods).

In [4]:
def run_prediction_pipeline(df):
    df_prophet = run_prophet(df)
    df_xgb, reg_xgb = run_xgboost(df_prophet)

    # Plot feature importance
    plt.figure(figsize=(8,6))
    plot_importance(reg_xgb, ax=plt.gca())
    plt.title("XGBoost Feature Importance")
    plt.show()

    # Create future dates (hourly for 5 years)
    # start from 1 january 2026
    future_dates = pd.date_range(
        start=pd.Timestamp('2026-01-01 00:00:00'),
        end=pd.Timestamp('2026-01-01 00:00:00') + pd.DateOffset(years=5),
        freq='h'
    )
    
    future_df = pd.DataFrame({'Date': future_dates})

    # Add required features (example, adjust as needed)
    future_df['HE'] = future_df['Date'].dt.hour

    # NERC holidays: New Year's Day, Memorial Day, Labor Day, Thanksgiving, and Christmas
    # add vacation indicator
    future_df['NERC_holiday'] = future_df['Date'].apply(
        lambda x: 1 if (x.month == 1 and x.day == 1) or
                         (x.month == 5 and x.day >= 25 and x.day <= 31 and x.weekday() == 0) or
                         (x.month == 9 and x.day >= 1 and x.day <= 7 and x.weekday() == 0) or
                         (x.month == 11 and x.month == 11 and x.day >= 22 and x.day <= 28 and x.weekday() == 3) or
                         (x.month == 12 and x.day == 25) else 0
    )
    
    # Peak hours 0 = off-peak, 1 = peak
    # Peak (P) hours = Mon-Fri, HE 7-22 excl NERC holidays
    future_df['P/OP'] = future_df.apply(
        lambda row: 1 if (row['Date'].weekday() < 5 and 7 <= row['Date'].hour <= 22 and row['NERC_holiday'] == 0) else 0,
        axis=1
    )

    future_df['day_of_week'] = future_df['Date'].dt.dayofweek
    future_df['is_weekend'] = future_df['day_of_week'].isin([5,6]).astype(int)

    # use Prophet to predict baseline generation
    prophet_data = future_df[['Date']].rename(columns={'Date':'ds'})
    forecast_future = Prophet(daily_seasonality=True, yearly_seasonality=True).fit(
        df[['Date','Gen']].rename(columns={'Date':'ds','Gen':'y'})
    ).predict(prophet_data)
    future_df['prophet_pred'] = forecast_future['yhat']

    # use XGBoost to predict residuals
    res_features = ['HE','day_of_week','is_weekend','P/OP']
    X_future = future_df[res_features]
    future_df['xgb_pred'] = reg_xgb.predict(X_future)
    future_df['final_pred'] = future_df['prophet_pred'] + future_df['xgb_pred']
    return future_df
    

In [5]:
data_dir = "../data"
caiso_historical_df = pd.read_csv(data_dir + "/CAISO-Historical-Data.csv")
prediction = run_prediction_pipeline(caiso_historical_df)

04:14:28 - cmdstanpy - INFO - Chain [1] start processing
04:14:29 - cmdstanpy - INFO - Chain [1] done processing


KeyError: "['HE'] not in index"

In [ ]:
prediction.to_csv(data_dir + "/CAISO-Future-Predictions.csv", index=False)

In [ ]:
# check how the percentage have negative values
negative_percentage = (prediction[prediction['final_pred'] < 0].shape[0] / prediction.shape[0]) * 100
negative_percentage

22.99600684540787

In [ ]:
prediction

,Date,HE,NERC_holiday,P/OP,day_of_week,is_weekend,prophet_pred,xgb_pred,final_pred
0,2026-01-01 00:00:00,0,1,0,3,0,31.372285,-33.208401,-1.836116
1,2026-01-01 01:00:00,1,1,0,3,0,24.600462,-33.208401,-8.607939
2,2026-01-01 02:00:00,2,1,0,3,0,9.276389,-33.208401,-23.932012
3,2026-01-01 03:00:00,3,1,0,3,0,-3.884887,-33.208401,-37.093288
4,2026-01-01 04:00:00,4,1,0,3,0,-7.413475,-33.208401,-40.621876
...,...,...,...,...,...,...,...,...,...
43820,2030-12-31 20:00:00,20,0,1,1,0,31.822641,-29.683079,2.139562
43821,2030-12-31 21:00:00,21,0,1,1,0,35.366422,-33.026001,2.340421
43822,2030-12-31 22:00:00,22,0,1,1,0,48.550344,-33.077694,15.472650
43823,2030-12-31 23:00:00,23,0,0,1,0,63.904278,-33.263466,30.640812
